In [1]:
jumper_data = [
        ("Tara Davis-Woodhall", 6.86, 'SSSSSSSSS'),
        ("Larissa Iapichino", 6.80, 'SSSSSSSSS'),
        ("Malaika Mihambo", 7.1, 'SSSSSSSSS'),
        ("Ese Brume", 6.59, 'SSSSSSSSS'),
        ("Ruth Usoro", 6.43, 'SSSSSSSSS'),
        ("Jasmine Moore", 6.75, 'SSSSSSSSS'),
        ("Prestina Ochonogor", 6.29, 'SSSSSSSSS'),
        ("Monae' Nichols", 6.57, 'SSSSSSSSS'),
        ("Alina Rotaru-Kottmann", 6.54, 'SSSSSSSSS'),
        ("Ackelia Smith", 6.44, 'SSSSSSSSS'),
        ("Marthe Koala", 6.52, 'SSSSSSSSS'),
        ("Hilary Kpatcha", 6.14, 'SSSSSSSSS'),
        ("Xiong Shiqi", 6.55, 'SSSSSSSSS'),
        ("Pauline Hondema", 5.83, 'SSSSSSSSS'),
        ("Fátima Diame", 6.45, 'SSSSSSSSS'),
        ("Ivana Španović", 6.41, 'SSSSSSSSS'),
        ("Chanice Porter", 6.35, 'SSSSSSSSS'),
        ("Milica Gardašević", 6.48, 'SSSSSSSSS'),
        ("Plamena Mitkova", 6.40, 'SSSSSSSSS'),
        ("Laura Raquel Müller", 6.30, 'SSSSSSSSS'),
        ("Petra Farkas", 6.32, 'SSSSSSSSS'),
        ("Natalia Linares", 6.22, 'SSSSSSSSS'),
        ("Eliane Martins", 6.36, 'SSSSSSSSS'),
        ("Agate de Sousa", 6.31, 'SSSSSSSSS'),
        ("Brooke Buschkuehl", 6.23, 'SSSSSSSSS'),
        ("Sumire Hata", 6.26, 'SSSSSSSSS'),
        ("Nikola Horowska", 6.31, 'SSSSSSSSS'),
        ("Mikaelle Assani", 6.00, 'SSSSSSSSS'),
        ("Esraa Owis", 6.08, 'SSSSSSSSS'),
        ("Tessy Ebosele", 6.06, 'SSSSSSSSS'),
        ("Lissandra Campos", 5.96, 'SSSSSSSSS')
    ]


In [3]:
# Final code
# Key result: Malaika CSS better win probability than SSS

import numpy as np
import pandas as pd

# Define the Jumper class
class Jumper:
    def __init__(self, jumper_number, jumper_name, standard_mean, std_dev=0.2, strategy='SSSSSSSSS'):
        self.jumper_number = jumper_number
        self.jumper_name = jumper_name
        self.standard_mean = standard_mean
        self.conservative_mean = standard_mean - 0.2
        self.std_dev = std_dev
        self.strategy = strategy
        self.conservative_scratch = 0.01
        self.standard_scratch = 0.2
        self.advance_count_a = 0
        self.advance_count_b = 0
        self.win_count = 0
        self.total_jumps = 0
        self.total_scratches = 0
        self.jumps_first = []
        self.jumps_a = []
        self.jumps_b = []

    def standard_jumps(self, num_jumps):
        return np.random.normal(self.standard_mean, self.std_dev, num_jumps)

    def conservative_jumps(self, num_jumps):
        return np.random.normal(self.conservative_mean, self.std_dev, num_jumps)

    def take_jump(self, jump_index):
        """Determines whether the jumper will take a conservative or standard jump and returns the result."""
        if self.strategy[jump_index] == 'C':
            if np.random.rand() > self.conservative_scratch:
                return self.conservative_jumps(1)[0]
            else:
                self.total_scratches += 1
                return np.nan
        else:
            if np.random.rand() > self.standard_scratch:
                return self.standard_jumps(1)[0]
            else:
                self.total_scratches += 1
                return np.nan


# Function to select top jumpers based on their best valid jumps
def select_top_jumpers(jumpers, jumps_attr, num_jumpers_to_advance):
    best_jumps = np.array([np.nanmax(getattr(jumper, jumps_attr)) for jumper in jumpers])
    valid_jumper_indices = np.where(~np.isnan(best_jumps))[0]
    top_jumpers_indices = valid_jumper_indices[np.argsort(best_jumps[valid_jumper_indices])[-min(num_jumpers_to_advance, len(valid_jumper_indices)):]]
    return top_jumpers_indices


def simulate_competition(num_simulations=10000, num_jumpers_to_round_a=12, num_jumpers_to_round_b=8, debug=False, jumper_data=None):
    if jumper_data is None:
        jumper_data = [
            ("Tara Davis-Woodhall", 6.86, 'SSSSSSSSS'),
            ("Larissa Iapichino", 6.80, 'SSSSSSSSS'),
            ("Malaika Mihambo", 7.1, 'SSSSSSSSS')
        ]

    jumpers = [
        Jumper(
            jumper_number=i + 1,
            jumper_name=name,
            standard_mean=mean,
            strategy=strategy
        ) for i, (name, mean, strategy) in enumerate(jumper_data)
    ]

    first_round_jumps = 3
    round_a_jumps = 3
    round_b_jumps = 3

    for simulation_num in range(1, num_simulations + 1):
        if debug:
            print(f"#### Simulation {simulation_num}:")
            print("Jumper Number | Jumper Name | Jump Strategy | First Round Jumps | Round A Jumps | Round B Jumps")

        # Round 1
        for jumper in jumpers:
            jumper.jumps_first.clear()
            jumper.jumps_a.clear()
            jumper.jumps_b.clear()
            for jump_index in range(first_round_jumps):
                jump = jumper.take_jump(jump_index)
                jumper.jumps_first.append(jump)
            jumper.total_jumps += len(jumper.jumps_first)

            if debug:
                print(f"{jumper.jumper_number} | {jumper.jumper_name} | {jumper.strategy} | {jumper.jumps_first} | {jumper.jumps_a} | {jumper.jumps_b}")


        top_jumpers_indices_a = select_top_jumpers(jumpers, 'jumps_first', num_jumpers_to_round_a)

        for index in top_jumpers_indices_a:
            jumpers[index].advance_count_a += 1

        # Round A
        for index in top_jumpers_indices_a:
            jumper = jumpers[index]
            for jump_index in range(round_a_jumps):
                jump = jumper.take_jump(jump_index + first_round_jumps)
                jumper.jumps_a.append(jump)
            jumper.total_jumps += len(jumper.jumps_a)

            if debug:
                print(f"{jumper.jumper_number} | {jumper.jumper_name} | {jumper.strategy} | {jumper.jumps_first} | {jumper.jumps_a} | {jumper.jumps_b}")


        top_jumpers_indices_b = select_top_jumpers([jumpers[i] for i in top_jumpers_indices_a], 'jumps_a', num_jumpers_to_round_b)
        top_jumpers_indices_b = [top_jumpers_indices_a[i] for i in top_jumpers_indices_b]

        for index in top_jumpers_indices_b:
            jumpers[index].advance_count_b += 1

        # Round B
        for index in top_jumpers_indices_b:
            jumper = jumpers[index]
            for jump_index in range(round_b_jumps):
                jump = jumper.take_jump(jump_index + first_round_jumps + round_a_jumps)
                jumper.jumps_b.append(jump)
            jumper.total_jumps += len(jumper.jumps_b)

            if debug:
                print(f"{jumper.jumper_number} | {jumper.jumper_name} | {jumper.strategy} | {jumper.jumps_first} | {jumper.jumps_a} | {jumper.jumps_b}")

        # Combine jumps from Rounds A and B for top jumpers and determine the winner
        all_jumps = [np.concatenate((jumpers[i].jumps_a, jumpers[i].jumps_b)) for i in top_jumpers_indices_b]
        all_jump_maxes = [np.nanmax(jumps) for jumps in all_jumps]
        winner = top_jumpers_indices_b[np.argmax(all_jump_maxes)]
        jumpers[winner].win_count += 1

   # Prepare data for output
    jumper_data = {
        "Jumper Number": [jumper.jumper_number for jumper in jumpers],
        "Jumper Name": [jumper.jumper_name for jumper in jumpers],  # New column for jumper names
        "Jump Strategy": [jumper.strategy for jumper in jumpers],  # Renamed column
        "Mean of Standard Jump (m)": [jumper.standard_mean for jumper in jumpers],
        "Mean of Conservative Jump (m)": [jumper.conservative_mean for jumper in jumpers],
        "Standard Deviation of Standard Jump (m)": [jumper.std_dev for jumper in jumpers],
        "Probability of Winning": [jumper.win_count / num_simulations for jumper in jumpers],  # Calculate win probabilities
        "Average Number of Jumps": [jumper.total_jumps / num_simulations for jumper in jumpers],  # Calculate average jumps
        "Average Number of Scratches": [jumper.total_scratches / num_simulations for jumper in jumpers],  # Calculate average scratches
        "Probability of Advancing to Round A": [jumper.advance_count_a / num_simulations for jumper in jumpers],  # Calculate advancing probabilities for Round A
        "Probability of Advancing to Round B": [jumper.advance_count_b / num_simulations for jumper in jumpers]  # Calculate advancing probabilities for Round B
    }


    return pd.DataFrame(jumper_data)



results_df = simulate_competition(num_jumpers_to_round_a=12, num_jumpers_to_round_b=8,jumper_data=jumper_data)

print(results_df.to_string(index=False))


<ipython-input-3-0a896f8dbd91>:51: RuntimeWarning: All-NaN axis encountered
  best_jumps = np.array([np.nanmax(getattr(jumper, jumps_attr)) for jumper in jumpers])


 Jumper Number           Jumper Name Jump Strategy  Mean of Standard Jump (m)  Mean of Conservative Jump (m)  Standard Deviation of Standard Jump (m)  Probability of Winning  Average Number of Jumps  Average Number of Scratches  Probability of Advancing to Round A  Probability of Advancing to Round B
             1   Tara Davis-Woodhall     SSSSSSSSS                       6.86                           6.66                                      0.2                  0.1009                   8.8152                       1.7811                               0.9795                               0.9589
             2     Larissa Iapichino     SSSSSSSSS                       6.80                           6.60                                      0.2                  0.0486                   8.7159                       1.7276                               0.9691                               0.9362
             3       Malaika Mihambo     SSSSSSSSS                       7.10                 